# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096821


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:15,  2.69s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:06,  2.44s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:47,  1.81s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:33,  1.33s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:12<00:00,  4.12it/s]

Rendering models:  93%|█████████▎| 28/30 [00:12<00:00,  3.99it/s]

Rendering models:  97%|█████████▋| 29/30 [00:12<00:00,  4.05it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  4.19it/s]

Jnursssmith5263                       0.056690
ellivc                                0.016348
jverderame                            0.006042
jnarayanbvg                           0.005055
not-logged-in-42aeec5653762437f6b7    0.000703
awright5                              0.002813
Quarkgc                               0.001031
framosuw                              0.000960
bryndismr                             0.000861
not-logged-in-c1baf067dee2ae342bf7    0.001725
Jessica_Korkmaz                       0.003372
not-logged-in-5f30b2bb9b7f9d49da41    0.004086
not-logged-in-314d48142a3a122ba666    0.001898
TRISTANM18                            0.000615
not-logged-in-214814e52004b42491a4    0.010169
acapirala                             0.000525
Lavadude                              0.031028
Mr_DMan                               0.001190
jordanrushworth                       0.002972
not-logged-in-5a859927574f86e26115    0.010467
Rootbert                              0.021128
flowers_in_ch

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())